في الاول هنعمل كريت ل الداتا ونخزنها في مونجو دي بي 

In [ ]:
from pymongo import MongoClient
import random

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"]

# ----- Prepare first and last names -----
first_names = ["Ahmed", "Ali", "Sara", "Mona", "Omar", "Youssef", "Nour", "Laila", "Khaled", "Hana"]
last_names = ["Refat", "Hassan", "Ibrahim", "Mohamed", "Farouk", "Adel", "Sami", "Nabil", "Tarek", "Fathy"]

# ----- Generate Users -----
users = []
for i in range(1, 21):  # 20 users
    user = {
        "user_id": i,
        "name": f"{random.choice(first_names)} {random.choice(last_names)}",
        "email": f"user{i}@example.com",
        "age": random.randint(18, 60)
    }
    users.append(user)

# Insert users
db["users"].insert_many(users)
print("Inserted 20 users")

# ----- Generate Products -----
products = []
categories = ["Electronics", "Books", "Clothes", "Sports"]
for i in range(1, 21):  # 20 products
    product = {
        "product_id": i,
        "name": f"Product {i}",
        "category": random.choice(categories),
        "price": round(random.uniform(10, 500), 2)
    }
    products.append(product)

# Insert products
db["products"].insert_many(products)
print("Inserted 20 products")







reviews = []
for i in range(1, 21):  # 20 reviews
    review = {
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is a comment for product {random.randint(1, 20)}"
    }
    reviews.append(review)

# Insert reviews
db["reviews"].insert_many(reviews)
print("Inserted 20 reviews")


orders = []
for i in range(1, 21):  # 20 orders
    order = {
        "order_id": i,
        "user_id": random.randint(1, 20),  # user_id موجود
        "products": [
            {"product_id": random.randint(1, 20), "quantity": random.randint(1, 5)}
            for _ in range(random.randint(1, 3))  # كل order فيها 1-3 منتجات
        ],
        "total": 0  # هنحسبها بعدين
    }
    # حساب total
    order["total"] = sum([p["quantity"] * round(random.uniform(10, 500), 2) for p in order["products"]])
    orders.append(order)

# Insert orders
db["orders"].insert_many(orders)
print("Inserted 20 orders")
orders = []




Inserted 20 users
Inserted 20 products


In [ ]:
#  create csv files 
import pandas as pd
import random

# table 1
users_sql = pd.DataFrame({
    "user_id": range(1, 21),
    "name": [f"User_{i}" for i in range(1, 21)],
    "email": [f"user{i}@sql.com" for i in range(1, 21)],
    "signup_date": pd.date_range("2025-01-01", periods=20)
})
users_sql.to_csv("users_sql.csv", index=False)


# table 2
products_sql = pd.DataFrame({
    "product_id": range(1, 21),
    "name": [f"Product_{i}" for i in range(1, 21)],
    "category": [random.choice(["Electronics", "Books", "Clothing"]) for _ in range(20)],
    "price": [round(random.uniform(10, 500), 2) for _ in range(20)]
})
products_sql.to_csv("products_sql.csv", index=False)

# table 3

orders_sql = []
for i in range(1, 21):
    orders_sql.append({
        "order_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "quantity": random.randint(1, 5)
    })

orders_sql_df = pd.DataFrame(orders_sql)
orders_sql_df.to_csv("orders_sql.csv", index=False)


# table 4

reviews_sql = []
for i in range(1, 21):
    reviews_sql.append({
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is SQL comment for product {random.randint(1, 20)}"
    })

reviews_sql_df = pd.DataFrame(reviews_sql)
reviews_sql_df.to_csv("reviews_sql.csv", index=False)


intrgation of data and the processing 
---------------------

In [1]:
# اول خطوه قراءه البيانات من mongodb
# create  sparksession 

from pymongo import MongoClient
import pandas as pd
from pyspark.sql   import SparkSession

# create  sparksession 

spark = (
    SparkSession.builder
    .appName("firt_ETL")
    .config("spark.jars", "/home/ahmed-refat/Desktop/end_to_end_projects/postgresql-42.2.5.jar")
    .getOrCreate()
)



# the connection of mongodb local 
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"] # دي اسم الداتا بيز بتاعتي في مونجو دي بي 

collection1= pd.DataFrame(list(db["users"].find())).drop(columns=["_id"])
collection2= pd.DataFrame(list(db["orders"].find())).drop(columns=["_id"])
collection3= pd.DataFrame(list(db["products"].find())).drop(columns=["_id"])




# collection قراءه ال 
df_users =spark.createDataFrame(collection1)
df_orders =spark.createDataFrame(collection2)
df_products =spark.createDataFrame(collection3)


df_users.show(5)
df_orders.show(5)
df_products.show(5)







Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/26 22:50:23 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/26 22:50:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/11/26 22:50:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+-------+-------------+-----------------+---+
|user_id|         name|            email|age|
+-------+-------------+-----------------+---+
|      1|  Ali Mohamed|user1@example.com| 38|
|      2|    Hana Sami|user2@example.com| 51|
|      3|   Ahmed Sami|user3@example.com| 28|
|      4|Khaled Hassan|user4@example.com| 18|
|      5|  Khaled Adel|user5@example.com| 29|
+-------+-------------+-----------------+---+
only showing top 5 rows


+--------+-------+--------------------+------------------+
|order_id|user_id|            products|             total|
+--------+-------+--------------------+------------------+
|       1|     14|[{quantity -> 4, ...|           4749.09|
|       2|     15|[{quantity -> 2, ...|            757.82|
|       3|     14|[{quantity -> 1, ...|           1215.49|
|       4|      7|[{quantity -> 3, ...|            827.63|
|       5|     12|[{quantity -> 5, ...|2769.6800000000003|
+--------+-------+--------------------+------------------+
only showing top 5 rows
+----------+---------+--------+------+
|product_id|     name|category| price|
+----------+---------+--------+------+
|         1|Product 1|   Books|438.83|
|         2|Product 2|   Books| 44.64|
|         3|Product 3| Clothes| 86.02|
|         4|Product 4|   Books|327.74|
|         5|Product 5|  Sports|335.79|
+----------+---------+--------+------+
only showing top 5 rows


In [2]:
# هنا بقا هنعمل عمليه تحويل الملفات csv  ل داتا فريم جوا باي سبارك 

# قراءة ملفات CSV في PySpark
df_users_csv = spark.read.csv("users_sql.csv", header=True, inferSchema=True)
df_products_csv = spark.read.csv("products_sql.csv", header=True, inferSchema=True)
df_orders_csv = spark.read.csv("orders_sql.csv", header=True, inferSchema=True)
df_reviews_csv = spark.read.csv("reviews_sql.csv", header=True, inferSchema=True)

# عرض أول 5 صفوف لأي ملف للتأكد

df_orders_csv.show(5)
df_reviews_csv.show(5)


+--------+-------+----------+--------+
|order_id|user_id|product_id|quantity|
+--------+-------+----------+--------+
|       1|      9|         8|       4|
|       2|      3|         7|       2|
|       3|      6|        16|       2|
|       4|      6|        14|       1|
|       5|      3|        18|       1|
+--------+-------+----------+--------+
only showing top 5 rows
+---------+-------+----------+------+--------------------+
|review_id|user_id|product_id|rating|             comment|
+---------+-------+----------+------+--------------------+
|        1|     11|        13|     2|This is SQL comme...|
|        2|      4|        11|     2|This is SQL comme...|
|        3|     11|         1|     2|This is SQL comme...|
|        4|      2|        13|     1|This is SQL comme...|
|        5|      8|         6|     4|This is SQL comme...|
+---------+-------+----------+------+--------------------+
only showing top 5 rows


In [3]:
# عمليه دمج الداتا فريمز المتشابها في داتا فريم واحده عن طريق حذف المكرر وعمل جوين عليهم 
csv_df_clean = df_users_csv.drop("name", "email")

df_users_clean = df_users.join(
    csv_df_clean,
    on="user_id",
    how="left"
)

# دمج جداول المنتجات 
# لما جيت اجرب لقيت الاتنين نفس الحاجه ف هنشتغل علي الجدول الي جي من مونجو دي بي وخلاص 
df_product_clean= df_products


# دمج جدولين ال orders 
# بس هيظهر قدمنا مشكله ان الجدول الي جي من المونجو فيه مشكله 
# وتحديدا ان في عمود اسمه برودكت جواه list 



df_orders.printSchema()
df_orders_csv.printSchema()



root
 |-- order_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- total: double (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [4]:
# create explode to the column of product in df_orders == mongodb
from pyspark.sql.functions import col , explode 
df_explode= (df_orders.withColumn("product",explode("products")))

fact_order=(
  df_explode
  .withColumn("product_id",col("product")["product_id"])
  .withColumn("quantity",col("product")["quantity"])
  .drop("product", "products")

)
#fact_order.show()


In [ ]:
# connect with postgres and add df_users_clean to dim_usres 
df_users_clean.write.jdbc(
    url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="dim_users",
    mode="append",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }
)

# connect with postgres and add df_products to dim_products 

df_products.write.jdbc(
   url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="dim_products",
    mode="append",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }    
)


git dim tables from  postgres to gir surrogate key and make join than load fact_orders  and fact_review 

In [5]:
df_users_dim_db = spark.read.jdbc(
    url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="dim_users",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }
)

df_products_dim_db = spark.read.jdbc(
    url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="dim_products",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }
)

#df_users_dim_db.show()
#df_products_dim_db.show()

# هنعمل جوين بقا لل fact مع الدايمنشنز التانيه 
fact_order_db = fact_order.join(
    df_users_dim_db.select("user_id", "user_key"),
    on="user_id",
    how="left"
).join(
    df_products_dim_db.select("product_id","product_key"),
    on="product_id",
    how="left"    
)

# هنختار الاعمده الي تتناسب اكتر مع الجدول الي صممناه في الداتا وير هاوس 
fact_order_final = fact_order_db.select(
    "user_key",
    "product_key",
    "order_id",
    "quantity",
    "total",
)
fact_order_final.show()


+--------+-----------+--------+--------+------------------+
|user_key|product_key|order_id|quantity|             total|
+--------+-----------+--------+--------+------------------+
|      17|         20|       4|       3|            827.63|
|      17|         15|       4|       1|            827.63|
|      17|         13|       4|       4|            827.63|
|       7|          2|       6|       4|           1428.96|
|       7|          8|       6|       2|           1428.96|
|      20|         11|       7|       2|            903.46|
|       2|          4|       5|       5|2769.6800000000003|
|       2|          8|       5|       3|2769.6800000000003|
|      18|         20|       9|       1|            2077.2|
|      18|          2|       9|       5|            2077.2|
|       1|         17|      10|       4|4072.8500000000004|
|       1|          1|      10|       5|4072.8500000000004|
|       4|         16|       1|       4|           4749.09|
|       4|         17|       1|       4|

In [ ]:
# connect with postgres and fact_order 

fact_order_final.write.jdbc(
    url="jdbc:postgresql://localhost:5432/ecommerce_dw",
    table="fact_orders",
    mode="append",
    properties={
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }
)